In [21]:
from yahoofinancials import YahooFinancials
from threading import Thread
import pandas as pd
from time import time

In [2]:
ticker=['AAPL','MSFT']
yahoo_financials = YahooFinancials(ticker)

balance_sheet_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'balance')
#income_statement_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'income')
#cash_statement_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'cash')
#all_statement_data_qt =  yahoo_financials.get_financial_stmts('quarterly', ['income', 'cash', 'balance'])
#earnings_data = yahoo_financials.get_stock_earnings_data()
#net_income = yahoo_financials.get_net_income()
#historical_stock_prices = yahoo_financials.get_historical_price_data('2008-09-15', '2018-09-15', 'weekly')

#### test

In [3]:
sheets = balance_sheet_data_qt['balanceSheetHistoryQuarterly']; len(sheets)

2

In [4]:
sheets.keys()

dict_keys(['AAPL', 'MSFT'])

In [5]:
d_up={}
for ticker in list(sheets.keys()):
    dates = [list(i.keys())[0] for i in sheets[ticker]]
    d={}
    for i in range(len(dates)):
        d[dates[i]] = sheets[ticker][i][dates[i]]
    d_up[ticker]=d
un_nested = {(outer_key, inner_key):inner_dict for (outer_key, outer_dict) in list(d_up.items()) for (inner_key, inner_dict) in outer_dict.items()}
balance = pd.DataFrame(un_nested)

In [6]:
balance.head()

AAPL                              \
                                2018-12-29    2018-09-29    2018-06-30   
accountsPayable               4.429300e+10  5.588800e+10  3.848900e+10   
cash                          4.477100e+10  2.591300e+10  3.197100e+10   
commonStock                   4.097000e+10  4.020100e+10  3.862400e+10   
deferredLongTermAssetCharges           NaN           NaN           NaN   
goodWill                               NaN           NaN           NaN   

                                                    MSFT                \
                                2018-03-31    2019-03-31    2018-12-31   
accountsPayable               3.431100e+10  7.544000e+09  7.563000e+09   
cash                          4.505900e+10  1.121200e+10  6.638000e+09   
commonStock                   3.804400e+10  7.779100e+10  7.755600e+10   
deferredLongTermAssetCharges           NaN           NaN           NaN   
goodWill                               NaN  4.186100e+10  4.157700e+10   

                                                         
                                2018-09-30   2018-06-30  
accountsPayable               8.511000e+09   8617000000  
cash                          1.513700e+10  11946000000  
commonStock                   7.130300e+10  71223000000  
deferredLongTermAssetCharges           NaN   1369000000  
goodWill                      3.585500e+10  35683000000

#### func

In [42]:
def yf_to_df(data):
    sheets = list(data.values())[0]
    d_up={}
    for ticker in list(sheets.keys()):
        dates = [list(i.keys())[0] for i in sheets[ticker]]
        d={}
        for i in range(len(dates)):
            d[dates[i]] = sheets[ticker][i][dates[i]]
        d_up[ticker]=d
    un_nested = {(outer_key, inner_key):inner_dict for (outer_key, outer_dict) in list(d_up.items()) for (inner_key, inner_dict) in outer_dict.items()}
    return pd.DataFrame(un_nested)

def get_fundamental_df(ticker,frequency='quarterly',statement_type='balance'):
    '''ticker: list of tickers or single ticker name'''
    yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_financial_stmts(frequency, statement_type)
    return yf_to_df(data)

def get_fundamental_df_multithread(ticker,frequency='quarterly',statement_type='balance'):
    '''
    ticker: list of tickers
    frequency: quarterly by default
    statement_type: balance by default
    '''   
    def get_for_multithread(ticker,frequency='quarterly',statement_type='balance'):
        nonlocal df
        yahoo_financials = YahooFinancials(ticker)
        data = yahoo_financials.get_financial_stmts(frequency, statement_type)
        df = pd.concat([df,yf_to_df(data)],axis=1,sort=True)
    
    threadlist=[]
    df=pd.DataFrame()
    for i in ticker:
        t = Thread(target=get_for_multithread,args=(i,frequency,statement_type))
        t.start()
        threadlist.append(t)
    for i in threadlist:
        i.join()
    return df

In [43]:
# time for multithreading
t0 = time()
balance = get_fundamental_df_multithread(['AAPL','MSFT','IBM','GE','XOM'])
t1 = time()
t1 - t0

9.0845468044281

In [44]:
balance.head()

AAPL                              \
                                2018-12-29    2018-09-29    2018-06-30   
accountsPayable               4.429300e+10  5.588800e+10  3.848900e+10   
cash                          4.477100e+10  2.591300e+10  3.197100e+10   
commonStock                   4.097000e+10  4.020100e+10  3.862400e+10   
deferredLongTermAssetCharges           NaN           NaN           NaN   
deferredLongTermLiab                   NaN           NaN           NaN   

                                                  XOM                \
                                2018-03-31 2019-03-31    2018-12-31   
accountsPayable               3.431100e+10        NaN  2.106300e+10   
cash                          4.505900e+10        NaN  3.042000e+09   
commonStock                   3.804400e+10        NaN  1.525800e+10   
deferredLongTermAssetCharges           NaN        NaN  3.209000e+09   
deferredLongTermLiab                   NaN        NaN           NaN   

                                                                  MSFT  \
                                2018-09-30    2018-06-30    2019-03-31   
accountsPayable               4.146600e+10  3.825000e+10  7.544000e+09   
cash                          5.669000e+09  3.430000e+09  1.121200e+10   
commonStock                   1.525400e+10  1.508600e+10  7.779100e+10   
deferredLongTermAssetCharges           NaN           NaN           NaN   
deferredLongTermLiab                   NaN           NaN           NaN   

                                                                        \
                                2018-12-31    2018-09-30    2018-06-30   
accountsPayable               7.563000e+09  8.511000e+09  8.617000e+09   
cash                          6.638000e+09  1.513700e+10  1.194600e+10   
commonStock                   7.755600e+10  7.130300e+10  7.122300e+10   
deferredLongTermAssetCharges           NaN           NaN  1.369000e+09   
deferredLongTermLiab                   NaN           NaN           NaN   

                                       IBM                            \
                                2019-03-31   2018-12-31   2018-09-30   
accountsPayable               5.711000e+09   6558000000   5384000000   
cash                          1.713400e+10  11379000000  11563000000   
commonStock                   5.528700e+10  55151000000  54987000000   
deferredLongTermAssetCharges  5.284000e+09   5216000000   4436000000   
deferredLongTermLiab          2.663000e+09   2676000000   2581000000   

                                                     GE                \
                               2018-06-30    2018-12-31    2018-09-30   
accountsPayable                5518000000  1.715300e+10  1.574800e+10   
cash                          10741000000  1.636900e+10  8.682000e+09   
commonStock                   54827000000  7.020000e+08  7.020000e+08   
deferredLongTermAssetCharges   4689000000  1.243200e+10  1.035400e+10   
deferredLongTermLiab           2613000000  5.739000e+09  3.515000e+09   

                                                          
                                2018-06-30    2018-03-31  
accountsPayable               1.502600e+10  1.506000e+10  
cash                          8.511000e+09  1.267900e+10  
commonStock                   7.020000e+08  7.020000e+08  
deferredLongTermAssetCharges  1.089100e+10  1.147900e+10  
deferredLongTermLiab          3.453000e+09  3.240000e+09

In [38]:
# time for non multithreading
t1 = time()
balance = get_fundamental_df(['AAPL','MSFT','IBM','GE','XOM'],statement_type='balance')
t1 = time()
t1 - t0

65.08449292182922

In [39]:
balance.head()

AAPL                              \
                                2018-12-29    2018-09-29    2018-06-30   
accountsPayable               4.429300e+10  5.588800e+10  3.848900e+10   
cash                          4.477100e+10  2.591300e+10  3.197100e+10   
commonStock                   4.097000e+10  4.020100e+10  3.862400e+10   
deferredLongTermAssetCharges           NaN           NaN           NaN   
deferredLongTermLiab                   NaN           NaN           NaN   

                                                    MSFT                \
                                2018-03-31    2019-03-31    2018-12-31   
accountsPayable               3.431100e+10  7.544000e+09  7.563000e+09   
cash                          4.505900e+10  1.121200e+10  6.638000e+09   
commonStock                   3.804400e+10  7.779100e+10  7.755600e+10   
deferredLongTermAssetCharges           NaN           NaN           NaN   
deferredLongTermLiab                   NaN           NaN           NaN   

                                                                   IBM  \
                                2018-09-30    2018-06-30    2019-03-31   
accountsPayable               8.511000e+09  8.617000e+09  5.711000e+09   
cash                          1.513700e+10  1.194600e+10  1.713400e+10   
commonStock                   7.130300e+10  7.122300e+10  5.528700e+10   
deferredLongTermAssetCharges           NaN  1.369000e+09  5.284000e+09   
deferredLongTermLiab                   NaN           NaN  2.663000e+09   

                                                                     \
                               2018-12-31   2018-09-30   2018-06-30   
accountsPayable                6558000000   5384000000   5518000000   
cash                          11379000000  11563000000  10741000000   
commonStock                   55151000000  54987000000  54827000000   
deferredLongTermAssetCharges   5216000000   4436000000   4689000000   
deferredLongTermLiab           2676000000   2581000000   2613000000   

                                        GE                              \
                                2018-12-31    2018-09-30    2018-06-30   
accountsPayable               1.715300e+10  1.574800e+10  1.502600e+10   
cash                          1.636900e+10  8.682000e+09  8.511000e+09   
commonStock                   7.020000e+08  7.020000e+08  7.020000e+08   
deferredLongTermAssetCharges  1.243200e+10  1.035400e+10  1.089100e+10   
deferredLongTermLiab          5.739000e+09  3.515000e+09  3.453000e+09   

                                                  XOM                \
                                2018-03-31 2019-03-31    2018-12-31   
accountsPayable               1.506000e+10        NaN  2.106300e+10   
cash                          1.267900e+10        NaN  3.042000e+09   
commonStock                   7.020000e+08        NaN  1.525800e+10   
deferredLongTermAssetCharges  1.147900e+10        NaN  3.209000e+09   
deferredLongTermLiab          3.240000e+09        NaN           NaN   

                                                          
                                2018-09-30    2018-06-30  
accountsPayable               4.146600e+10  3.825000e+10  
cash                          5.669000e+09  3.430000e+09  
commonStock                   1.525400e+10  1.508600e+10  
deferredLongTermAssetCharges           NaN           NaN  
deferredLongTermLiab                   NaN           NaN

In [14]:
income = get_fundamental_df(['AAPL','MSFT'],statement_type='income'); income.head()

AAPL                              \
                             2018-12-29    2018-09-29    2018-06-30   
costOfRevenue              5.227900e+10  3.881600e+10  3.284400e+10   
discontinuedOperations              NaN           NaN           NaN   
ebit                       2.334600e+10  1.611800e+10  1.261200e+10   
effectOfAccountingCharges           NaN           NaN           NaN   
extraordinaryItems                  NaN           NaN           NaN   

                                                 MSFT                \
                             2018-03-31    2019-03-31    2018-12-31   
costOfRevenue              3.771500e+10  1.017000e+10  1.242300e+10   
discontinuedOperations              NaN           NaN           NaN   
ebit                       1.589400e+10  1.034100e+10  1.025800e+10   
effectOfAccountingCharges           NaN           NaN           NaN   
extraordinaryItems                  NaN           NaN           NaN   

                                                       
                             2018-09-30    2018-06-30  
costOfRevenue              9.905000e+09  9.742000e+09  
discontinuedOperations              NaN           NaN  
ebit                       9.955000e+09  1.037900e+10  
effectOfAccountingCharges           NaN           NaN  
extraordinaryItems                  NaN           NaN

In [13]:
cash = get_fundamental_df(['AAPL','MSFT'],statement_type='cash'); cash.head()

AAPL                              \
                              2018-12-29    2018-09-29    2018-06-30   
capitalExpenditures        -3.355000e+09 -3.041000e+09 -3.267000e+09   
changeInCash                1.885800e+10 -6.058000e+09 -1.308800e+10   
changeToAccountReceivables  5.130000e+09 -9.078000e+09  2.330000e+08   
changeToInventory          -1.076000e+09  1.942000e+09  1.693000e+09   
changeToLiabilities        -8.871000e+09  2.037300e+10  1.500000e+09   

                                                  MSFT                \
                              2018-03-31    2019-03-31    2018-12-31   
capitalExpenditures        -4.195000e+09 -2.565000e+09 -3.707000e+09   
changeInCash                1.756800e+10  4.574000e+09 -8.499000e+09   
changeToAccountReceivables  9.093000e+09  4.600000e+08 -2.396000e+09   
changeToInventory          -3.241000e+09  1.200000e+07  1.654000e+09   
changeToLiabilities        -2.812100e+10 -1.770000e+08 -2.562000e+09   

                                                      
                              2018-09-30  2018-06-30  
capitalExpenditures        -3.602000e+09 -3980000000  
changeInCash                3.191000e+09  2725000000  
changeToAccountReceivables  9.194000e+09 -9188000000  
changeToInventory          -9.560000e+08  -572000000  
changeToLiabilities        -2.836000e+09  9712000000